In [ ]:
# docker pull emacski/tensorflow-serving:latest-linux_arm64
# docker run -t --rm -p 8501:8501 --mount type=bind,source=/tmp/model_name/,target=/models/model_name/ -e MODEL_NAME=model_name emacski/tensorflow-serving:latest-linux_arm64

In [86]:
#!docker start #como iniciar da  linha de comando
!open --background -a Docker

In [87]:
!docker run hello-world


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (arm64v8)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



In [33]:
#!docker pull tensorflow/serving

In [95]:
# esse funciona para quem n tem m1
#!docker pull tensorflow/serving:latest-gpu

In [22]:
# conda install tensorflow

In [9]:
import tensorflow as tf

In [14]:
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B0, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np

In [15]:
model = EfficientNetV2B0(weights='imagenet')

In [12]:
pwd

'/Users/csamp/Documents/stack_ab'

In [23]:
# img_path = 'tf_serving/lion.jpg'
img_path = 'tf_serving/pexels-artūras-kokorevas-10547480.jpg'

resized_height, resized_width = (250, 250)
img = image.load_img(img_path, target_size=(height, width))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)

print('Predicted:', decode_predictions(preds, top=3)[0])

1/1 [==============================] - 0s 28ms/step
Predicted: [('n09332890', 'lakeside', 0.2955897), ('n09421951', 'sandbar', 0.24374594), ('n01855672', 'goose', 0.10379495)]


In [81]:
import time
current_time = int(time.time())
path = f"effv2b0/{current_time}"
model.save(path)

INFO:tensorflow:Assets written to: effv2b0/1670550215/assets


INFO:tensorflow:Assets written to: effv2b0/1670550215/assets


In [82]:
path

'effv2b0/1670550215'

In [3]:
# # how the folder should look like
# ├── effv2b0
# │ ├── 1670405165
# │ │ ├── assets
# │ │ ├── saved_model.pb
# │ │ └── variables

In [26]:
# esse funciona
#!docker pull emacski/tensorflow-serving:latest-linux_arm64

latest-linux_arm64: Pulling from emacski/tensorflow-serving
Digest: sha256:37ae953d10a10f520fc7aca068c866f28a424894c47b1d06c8cec22e631e29be
Status: Image is up to date for emacski/tensorflow-serving:latest-linux_arm64
docker.io/emacski/tensorflow-serving:latest-linux_arm64


In [94]:
# !docker run -t --rm -p 8501:8501 \
#     -v "/Users/csamp/Documents/stack_ab/effv2b0/:/models/effv2b0/" \
#     -e MODEL_NAME=effv2 \
#     emacski/tensorflow-serving:latest-linux_arm6

In [50]:
#!docker run -p 8501:8501 --name tfserving_vgg16 --mount type=bind,source=vgg16/,target=vgg16/ -e MODEL_NAME=vgg16 -t tensorflow/serving

In [87]:
#!docker run -p 8502:8502 --name ttfserving_vgg -v source=/Users/csamp/Documents/stack_ab/vgg16/,target=vgg16 -e MODEL_NAME=vgg16 -t emacski/tensorflow-serving:latest-linux_arm64

In [13]:
pwd

'/Users/csamp/Documents/stack_ab'

In [88]:
#docker pull tensorflow/serving
# esse funciona
!docker run --rm -p 8501:8501 \
        --name tfserving_effv2 \
        -v "/Users/csamp/Documents/stack_ab/effv2b0:/models/effv2b0" \
        -e MODEL_NAME=effv2b0 \
        emacski/tensorflow-serving:latest-linux_arm64

2022-12-09 01:44:49.693292: I external/tf_serving/tensorflow_serving/model_servers/server.cc:89] Building single TensorFlow model file config:  model_name: effv2b0 model_base_path: /models/effv2b0
2022-12-09 01:44:49.694751: I external/tf_serving/tensorflow_serving/model_servers/server_core.cc:465] Adding/updating models.
2022-12-09 01:44:49.694764: I external/tf_serving/tensorflow_serving/model_servers/server_core.cc:591]  (Re-)adding model: effv2b0
2022-12-09 01:44:49.805917: I external/tf_serving/tensorflow_serving/core/basic_manager.cc:740] Successfully reserved resources to load servable {name: effv2b0 version: 1670550215}
2022-12-09 01:44:49.805953: I external/tf_serving/tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: effv2b0 version: 1670550215}
2022-12-09 01:44:49.805965: I external/tf_serving/tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: effv2b0 version: 1670550215}
2022-12-09 01:44:49.806719: I external/

In [89]:
# API endpoint can be accessed at http://localhost:8501. 

In [90]:
import json
import requests

def run_prediction(data,headers,endpoint):
    json_response = requests.post(endpoint,data=data,headers=headers)
    prediction = json.loads(json_response.text)
    return (decode_predictions(np.array(preds), top=3)[0])

In [ ]:
json.loads(json_response.text)

In [104]:
endpoint = 'http://localhost:8501/v1/models/effv2:predict'
data = json.dumps({"instances": x.tolist()})
headers = {"content-type": "application/json"}
run_prediction(data,headers,endpoint)

[('n09332890', 'lakeside', 0.2955897),
 ('n09421951', 'sandbar', 0.24374594),
 ('n01855672', 'goose', 0.10379495)]

In [103]:
# code from https://cnvrg.io/how-to-serve-a-model-with-tensorflow/
# another way: https://medium.com/analytics-vidhya/deploy-production-ready-keras-model-with-tensorflow-serving-and-docker-85615245a9ed

In [ ]:
json_response = requests.post('http://localhost:8501/v1/models/effv2:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)
print('Predicted:', decode_predictions(preds, top=3)[0])

In [59]:
# ran docker with
# docker run -t --rm -p 8501:8501 \
#     -v "/Users/csamp/Documents/stack_ab/vgg16/:/models/vgg16/" \
#     -e MODEL_NAME=vgg16 \
#     emacski/tensorflow-serving:latest-linux_arm64

I ran docker with:

`
docker run -t --rm -p 8501:8501 \
    -v "/Users/csamp/Documents/stack_ab/vgg16/:/models/vgg16/" \
    -e MODEL_NAME=vgg16 \
    emacski/tensorflow-serving:latest-linux_arm64`
`

docker run -t --rm -p 8501:8501 \ 
    --mount type=bind,source=/tmp/model_name/,target=/models/model_name/ 
    -e MODEL_NAME=model_name \ 
    emacski/tensorflow-serving:latest-linux_arm64

In [102]:
!docker run -t --rm -p 8501:8501 \
    -v "/Users/csamp/Documents/stack_ab/effv2b0/:/models/effv2b0/" \
    -e MODEL_NAME=effv2b0 \
    emacski/tensorflow-serving:latest-linux_arm64

2022-12-09 01:49:20.549439: I external/tf_serving/tensorflow_serving/model_servers/server.cc:89] Building single TensorFlow model file config:  model_name: effv2b0 model_base_path: /models/effv2b0
2022-12-09 01:49:20.549712: I external/tf_serving/tensorflow_serving/model_servers/server_core.cc:465] Adding/updating models.
2022-12-09 01:49:20.549726: I external/tf_serving/tensorflow_serving/model_servers/server_core.cc:591]  (Re-)adding model: effv2b0
2022-12-09 01:49:20.657382: I external/tf_serving/tensorflow_serving/core/basic_manager.cc:740] Successfully reserved resources to load servable {name: effv2b0 version: 1670550215}
2022-12-09 01:49:20.657418: I external/tf_serving/tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: effv2b0 version: 1670550215}
2022-12-09 01:49:20.657436: I external/tf_serving/tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: effv2b0 version: 1670550215}
2022-12-09 01:49:20.657975: I external/